In [ ]:
## KNN ##
# Classify new data points based on distance to know data
# Define distance metric between items in dataset and find K closest items
# Then use these items to predict some test item's property by voting on it

In [1]:
# KNN on MovieLens. Predict rating using 10 similar movie's genre + popularity (total number of ratings)
import pandas as pd
import numpy as np

rating_cols = ['user_id','movie_id','rating']
ratings = pd.read_csv(r'C:\Users\MHayden\Desktop\ml-100k\u.data',sep='\t',names=rating_cols,usecols=range(3),encoding="ISO-8859-1")
ratings.head()

,user_id,movie_id,rating
0,0,50,5
1,0,172,5
2,0,133,1
3,196,242,3
4,186,302,3


In [2]:
# Group by movie ID. Compute each movies (popularity) and average rating
movieProperties = ratings.groupby('movie_id').agg({'rating':[np.size,np.mean]})
movieProperties.head()

rating          
           size      mean
movie_id                 
1           452  3.878319
2           131  3.206107
3            90  3.033333
4           209  3.550239
5            86  3.302326

In [ ]:
# Stopped at 1:22